In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.image import ImageDataGenerator
import cv2 #opencv+python == 4.9.0.80
import numpy as np

Data Loading/Preprocessing/Image Normalizing

In [ ]:
# we want to normalize pixel values
scaler = MinMaxScaler()

# looping through files to call the scaling/ alt way for this line: image = scaler.fit_transform(image.reshape(-1, 1)).reshape(image.shape)
# for i in range(images.shape[0]):
#    images[i, :, :] = scaler.fit_transform(images[i, :, :].reshape(-1, 1)).reshape(images[i, :, :].shape)

In [ ]:
# looping both i and file (2 vars)
# start at 1 bc data files starts at 1
for i, file in enumerate(files, start=1):
    if i % 10 == 0: # we can check the loading progress by 10s as % (like a loading bar for visualizing)
        sys.stdout.write('\r[{}/{}] images loaded: {:.1f} %'.format(i, len(files), i / float(len(files)) * 100))
        sys.stdout.flush() # "flush" good to add, it makes progress immediately visible instead of waiting for compiling 

In [ ]:
# before minmaxing we want to resize the images (w Cv2)
# 1. resize 2. store for later use in script
    mat_file = hdf5storage.loadmat(os.path.join(data_dir, file))['cjdata'][0]
    image = cv2.resize(mat_file[2], dsize=(args.image_dimension, args.image_dimension), # default is usually 512, but args.image_dimension is a command line tool that will select the dimension for us (resize itself)
                       interpolation=cv2.INTER_CUBIC)
    mask = cv2.resize(mat_file[4].astype('uint8'), dsize=(args.image_dimension, args.image_dimension), # same for mask as for image
                      interpolation=cv2.INTER_CUBIC)

image = scaler.fit_transform(image.reshape(-1, 1)).reshape(image.shape) # here we apply the scalar, and we transposed the image

Histogram Equalizer (method 1)

In [ ]:
image = cv2.equalizeHist((image * 255).astype(np.uint8))/255.0 
# each pixel value gets x by 255
# why uint8? to cast as a type that is unassigned (no +/-), (common when working with image intensity)
# need the .0 bc equalizer spits out float, so to help it)

Contrast Adjustment (or method 2)

In [ ]:
# alpha = 1.5 # remember alpha is multiplied by source
# beta = 10 # remember beta is added to source
# image = cv2.convertScaleAbs(image, alpha, beta) # src = image, alpha = contrast, beta = brightness, convert = saturate the image
# # we're taking in the source, alpha, beta and multiplying the 
# # increase brightness b > 0, decrease b < 0
# # increase contrast a > 1 (cant multiply by 0!), or decrease a < 1, 

In [ ]:
labels.append(int(mat_file[0])) # fixing our labels because of the transpose 
images.append(image) # appending the resized images to the variable of the initial images
masks.append(mask.astype(bool)) # aka booleans only, is there a mask or not? 

In [ ]:
labels = np.array(labels) # converting labels to numpy arrays
images = np.array(images) #""
masks = np.array(masks) #""

Apply data normalizaiton to the whole dataset

In [ ]:
## following chunk of code is run after splitting data into both training and testing ##
images_train = scaler.fit_transform(images_train.reshape(-1, 1)).reshape(images_train.shape)
images_cv = scaler.fit_transform(images_cv.reshape(-1, 1)).reshape(images_cv.shape)
images_test = scaler.fit_transform(images_test.reshape(-1, 1)).reshape(images_test.shape)

Data augmentation (for generalizability, to potentially help the accuracy score)

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2, # equal force from opposite directions
    zoom_range=0.2, # zooming through the image
    horizontal_flip=True, # mirror it
    fill_mode='nearest' 
)

# train the model using training data generated and augmented parameters
model.fit(datagen.flow(images_train, masks_train, batch_size=2), epochs=30, verbose=1, validation_data=(images_cv, masks_cv), callbacks=[checkpoint])